In [1]:
import sumolib

In [1]:
net = sumolib.net.readNet('sumo_stuff/Grid.net.xml')

NameError: name 'sumolib' is not defined

In [3]:
# get the list of edges
edge_list = net.getEdges()

# get the list of nodes
node_list = net.getNodes()

print(f"The road network has {len(edge_list)} edges and {len(node_list)} nodes.")

The road network has 360 edges and 100 nodes.


In [ ]:
def generate_random_trips(net:sumolib.net.Net, n_trips:int=100, min_edges:int=3):
    generated_trips = []
    while len(generated_trips) < n_trips:
        trip = net.get
    

In [11]:
!python "C:\Program Files (x86)\Eclipse\Sumo\tools\randomTrips.py" -n sumo_stuff\Grid.net.xml -e 100

## Real World

In [2]:
import traci
import sys
import os

def init_traci(config_file_path):
    
    if 'SUMO_HOME' in os.environ:
        tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
        sys.path.append(tools)
    else:
        sys.exit("please declare environment variable 'SUMO_HOME'")

    #Configuration
    sumo_binary = os.environ['SUMO_HOME']+"/bin/sumo"

    sumo_cmd = [sumo_binary, "-c", config_file_path]

    traci.start(sumo_cmd)
    traci.simulationStep()

In [3]:
def simulate(n_steps, path):
    init_traci(path)
    data = [] # vehicle_id, lon, lat, time, vClass
# simulate each step
    for step in range(n_steps):
        
        # perform a simulation step
        traci.simulationStep()
        
        # get the list of active vehicles (vehicles inserted in the simulation)
        vehicle_list = traci.vehicle.getIDList()
        
        time = traci.simulation.getTime()
        
        # value retrieval
        if len(vehicle_list) == 0:
            print("Simulation ended at step #"+str(step))
            break
            
        for v_id in vehicle_list:
            v_class = traci.vehicle.getVehicleClass(v_id)
            
            # Position
            x, y = traci.vehicle.getPosition(v_id)
            lon, lat = traci.simulation.convertGeo(x, y)
            
            data += [
                (v_id, lon, lat, time, v_class)
            ]
    traci.close()
    
    return data

In [4]:
"""import pandas as pd

path = "sumo_stuff/Grid-Empire-2.5km/osm.sumocfg"
n_steps = 60 * 60 # min

data = simulate(n_steps, path)

df = pd.DataFrame(data, columns=["vehicle_id", "lon", "lat", "time", "class"]).sort_values(["vehicle_id", "time"])

df"""

'import pandas as pd\n\npath = "sumo_stuff/Grid-Empire-2.5km/osm.sumocfg"\nn_steps = 60 * 60 # min\n\ndata = simulate(n_steps, path)\n\ndf = pd.DataFrame(data, columns=["vehicle_id", "lon", "lat", "time", "class"]).sort_values(["vehicle_id", "time"])\n\ndf'

In [5]:
from glob import glob
import pandas as pd

n_steps = 60 * 60 # min

for file in glob("sumo_stuff/*/osm.sumocfg"):
    save_path = file[:-11] + "trajectories.zip"
    print(save_path)
    data = simulate(n_steps, file)

    df = pd.DataFrame(data, columns=["vehicle_id", "lon", "lat", "time", "class"]).sort_values(["vehicle_id", "time"])
    
    df.to_csv(save_path, index=False)
    

sumo_stuff\Borgo-Rome-2Km\trajectories.zip
sumo_stuff\Grid-Empire-2.5km\trajectories.zip
sumo_stuff\Mixed-Athens-2km\trajectories.zip
